# Raw Data Processing

## 1. Web crawler and flux sites htm data preprocessing
For 2017 version of this paper builds, the resource of site information must be reliable, and only two websites are selected for web crawling: The new Fluxnet site and old Fluxnet ORNL site. In case that the wegpages are rather different,we chose Scrapy to download all the pages to local disk, and Beautifulsoup to extract information from webpages.

### 1.1 Web crawler (Scrapy) deployment

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

class FluxSpider(scrapy.Spider):
    name = "fluxnet"
    allowed_domains = ["fluxnet.ornl.gov", "fluxnet.fluxdata.org", "fluxdata.org", "ameriflux.lbl.gov", "asiaflux.net", "europe-fluxdata.eu","data.ozflux.org.au"]
    start_urls = [
        "https://fluxnet.ornl.gov/site_list/sitename/-",          # ORNL Database Site List
        "http://fluxnet.fluxdata.org/sites/site-list-and-pages/"  # LUXNET2015 Dataset Site List
        "http://www.fluxdata.org:8080/SitePages/"                 # La Thuile Synthesis Dataset Site List
        "http://ameriflux.lbl.gov/sites/site-list-and-pages/"     # Ameriflux Site List
        "http://asiaflux.net/?page_id=22"                         # Asianflux Site List
        "http://www.europe-fluxdata.eu/home/sites-list"           # European Flux Database Cluster Site List
        "http://data.ozflux.org.au/portal/pub/listPubCollections.jspx?pageNo=1&sizePerPage=200" #OzFlux Site List
                ]

    def parse(self, response):
        filename = response.url.split("/")[-1]
        with open(filename, 'wb') as f:
            f.write(response.body)

#Initiate the Spider in Python Console
settings = get_project_settings()
process = CrawlerProcess(settings=settings)
process.crawl(FluxSpider)
process.start()
#Crawl results are regrouped into FluxNet_Old_ORNL and FLuxnet_2015Datasets

### 1.2 Data extraction and cleaning using Beautifulsoup
Beautifulsoup is quite efficient in extracting information form tabled webpages, we just inspected the source code of one page to seek for pattern, then applied this pattern to extraction codes, which would automatically deliever the values into right fields. There were bunches of webpages coming from different sites, so we extract them seperately, and combined them together. Few 'br/'s exist in Investigators field, just define a function to delelte them. Let see how the table looks like.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

def ReturnList(Pos):
    filelist = []
    for root, dirs, files in os.walk(Pos):
        for name in files:
            filelist.append(os.path.join(root, name))
    return(filelist)

#Define the positions of webpages
FluxORNLPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/FluxNet_Old_ORNL/'
FluxPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/FLuxnet_2015Datasets/'
LaThuilePos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/LaThuile_SitePages/'
AmeriPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/AmeriFlux/'
AsiaPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/AsiaFlux/'
EFDCPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/EFDC/'
OzPos = 'D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/OzFlux/'

#ORNL Fluxnet site
htmllist = ReturnList(FluxORNLPos)
htmltable = []
#Parser for extraction
for html in htmllist:
    htmlmarker = open(html)
    soup = BeautifulSoup(htmlmarker,'lxml')
    tables = soup.find_all('table') #Locate all tables
    Fluxinfo = tables[0].find_all('td')#Extract each table
    Locinfo = tables[1].find_all('td')
    Investinfo = tables[2].find_all('td')
    temp = {}
    #First td tab processing
    temp.update({'SiteName':Fluxinfo[1].contents})
    temp.update({'Description':Fluxinfo[3].contents})
    temp.update({'Code':Fluxinfo[7].contents})
    #Second
    temp.update({'Country':Locinfo[1].contents})
    temp.update({'Coordinates':Locinfo[3].contents})
    temp.update({'Data_Availability':0})
    #Third
    Investors = []
    for i in range(len(Investinfo)):
        if '@' in str(Investinfo[i]):
            Investors.append(Investinfo[i].contents[0])
    temp.update({'Investigators':Investors})
    htmltable.append(temp)
#    print 'Current Processing ORNL sites:', len(htmltable)
htmlmarker.close()

#2015DataSets Sites
htmllist = ReturnList(FluxPos)
Newhtmltable = []
#Parser for extraction
for html in htmllist:
    htmlmarker = open(html)
    soup = BeautifulSoup(htmlmarker,'lxml')
    tables = soup.find_all('table') # Locate all tables
    Fluxinfo = tables[0].find_all('td') # Only one table left in New Files 
    temp = {}
    temp.update({'SiteName':Fluxinfo[4].contents})
    temp.update({'Code':Fluxinfo[2].contents})
    temp.update({'Coordinates':(Fluxinfo[8].contents,Fluxinfo[10].contents)})
    
    data_avail = ''
    for item in Fluxinfo:
        if 'Duration' in str(item):
            data_avail = str(item).split('  ')[-1].split(' years')[0]
    temp.update({'Data_Availability':int(data_avail)})    
    
    Investors = []
    for Inves in Fluxinfo[6].find_all('a'):
        InvestName = Inves.contents[0].split(' <')[0]
        Investors.append(InvestName)
    temp.update({'Investigators':Investors})
    Newhtmltable.append(temp)
#    print 'Current Processing 2015 Datasets sites:', len(Newhtmltable)
htmlmarker.close()

#LaThuile_Site
htmllist = ReturnList(LaThuilePos)
LaThuhtmltable = []
#Parser for extraction
for html in htmllist:
    htmlmarker = open(html)
    soup = BeautifulSoup(htmlmarker,'lxml')
    tables = soup.find_all('table') # Locate all tables
    Fluxinfo = tables[0].find_all('td')# Only one table left in New Files 
    temp = {}
    temp.update({'SiteName':Fluxinfo[4].contents})
    temp.update({'Code':Fluxinfo[2].contents})
    temp.update({'Coordinates':(Fluxinfo[8].contents,Fluxinfo[10].contents)})
    temp.update({'Network':Fluxinfo[14].contents})
    
    data_avail = [Fluxinfo[24].contents[0].split(': ')[1].split(' (')[0], Fluxinfo[24].contents[2].split(': ')[1].split(' (')[0]]
    mx = max(int(data_avail[0]),int(data_avail[1]))
    temp.update({'Data_Availability':mx})
    
    Investors = []
    for Inves in Fluxinfo[6].find_all('a'):
        InvestName = Inves.contents[0].split(' -')[0] 
        Investors.append(InvestName)
    temp.update({'Investigators':Investors})
    LaThuhtmltable.append(temp)
#    print 'Current Processing LaThu Datasets sites:', len(LaThuhtmltable)
htmlmarker.close()

#Regional Networks
htmllist = ReturnList(AmeriPos + AsiaPos + EDFCPos + OzPos)
Regionaltable = []
#Parser for extraction
for html in htmllist:
    htmlmarker = open(html)
    soup = BeautifulSoup(htmlmarker,'lxml')
    tables = soup.find_all('table') # Locate all tables
    Fluxinfo = tables[0].find_all('td')# Only one table left in New Files 
    temp = {}
    temp.update({'SiteName':Fluxinfo[4].contents})
    temp.update({'Code':Fluxinfo[2].contents})
    temp.update({'Coordinates':(Fluxinfo[8].contents,Fluxinfo[10].contents)})
    temp.update({'Network':Fluxinfo[14].contents})
    
    data_avail = [Fluxinfo[24].contents[0].split(': ')[1].split(' (')[0], Fluxinfo[24].contents[2].split(': ')[1].split(' (')[0]]
    mx = max(int(data_avail[0]),int(data_avail[1]))
    temp.update({'Data_Availability':mx})
    
    Investors = []
    for Inves in Fluxinfo[6].find_all('a'):
        InvestName = Inves.contents[0].split(' -')[0] 
        Investors.append(InvestName)
    temp.update({'Investigators':Investors})
    Regionaltable.append(temp)
#    print 'Current Processing LaThu Datasets sites:', len(LaThuhtmltable)
htmlmarker.close()

Full_Table = htmltable + Newhtmltable + LaThuhtmltable + Regionaltable
Full_Table = sorted(Full_Table, key = lambda item: item['Code'][0])
x = pd.DataFrame.from_dict(Full_Table)

#Create a New List without duplicated sites
def CompareItem(item, temp):
    if item > temp:
        return(item)
    else:
        return(temp)
    
NewDict = []
for index, item in x.iterrows():
    if index == 0:
        temp = item
        continue
#    print 'Current processing item:', index
    if item.Code == temp.Code:
        temp.Country = CompareItem(item.Country, temp.Country) #Check Country
        temp.Coordinates = CompareItem(item.Coordinates,temp.Coordinates) #Check Coordiantes
        temp.Investigators = CompareItem(item.Investigators,temp.Investigators) #Check Investigators
        temp.Network = CompareItem(item.Network,temp.Network) #Check Network
        temp.SiteName = CompareItem(item.SiteName,temp.SiteName) #Check Network
        temp.Data_Availability = CompareItem(item.Data_Availability,temp.Data_Availability)
    else:
        NewDict.append({'Code':temp.Code, 'Country':temp.Country, 'Coordinates':temp.Coordinates, 'Investigators':temp.Investigators , 'Network':temp.Network, 'SiteName':temp.SiteName, 'Data_Avail':temp.Data_Availability})
        temp = item
#Append the last record to the dict
NewDict.append({'Code':temp.Code, 'Country':temp.Country, 'Coordinates':temp.Coordinates, 'Investigators':temp.Investigators , 'Network':temp.Network, 'SiteName':temp.SiteName, 'Data_Avail':temp.Data_Availability})


All_Sites_Description = pd.DataFrame.from_dict(NewDict)
All_Sites_Description.to_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/Full_SiteInfo.csv')

In [2]:
All_Sites_Description

,Code,Coordinates,Country,Data_Avail,Investigators,Network,SiteName
0,[AQ-KSe],"[-62.232115, -58.804321]",[Antarctica],0,[Taejin Choi],NaN,[King Sejong Station site ]
1,[AR-Lac],"([-29.2640], [-61.0280])",[Argentina],0,[Chiara Crotti],[Unaffiliated],[La Ciguena – Santa Fe]
2,[AR-SLu],"([-33.4648], [-66.4598])",NaN,3,[Gabriela Posse],[Unaffiliated],[San Luis]
3,[AR-Vir],"([-28.2395], [-56.1886])",NaN,4,[Gabriela Posse],[Unaffiliated],[Virasoro]
4,[AT-Fue],"[47.367374, 11.854709]",[Austria],0,[Georg Wohlfahrt],NaN,[Fuegen]
5,[AT-Hnb],"[47.803333, 13.591389]",[Austria],0,[Bradley Matthews],NaN,[Höllengebirge]
6,[AT-Lan],"([47.0612], [10.9635])",[Austria],0,[Georg Wohlfahrt],[Unaffiliated],[Langenfeld]
7,[AT-Leu],"([47.3780], [11.1628])",[Austria],0,[Georg Wohlfahrt],[Unaffiliated],[Leutasch]
8,[AT-Neu],"([47.1167], [11.3175])",[Austria],5,[Georg Wohlfahrt],"[GHG-Europe, CarboEuropeIP]",[Neustift/Stubai Valley]
9,[AT-Rtz],"([47.3947], [11.8047])",[Austria],0,[Georg Wohlfahrt],[],[Rotholz]


Now we are almost there, the next job is to extract investigators and their respective sites, bring them together and save this information into a list. As this list is still a Site -> Investigators list, we will transform it into a Investigator -> Sites list, together with the Data_Year attribute.

In [3]:
#Combine Investors and Sites
Site_Invest = []
for site in NewDict:
    for invest in site['Investigators']:
        Site_Invest.append((site['Code'][0], invest, site['Data_Avail'])) #Here <a is just the head of human page introduction

#Create Investor List and Remove Duplicate
InvestList = []
for Site in Site_Invest:
    Investor = Site[1]
    if Investor not in InvestList:
        InvestList.append(Investor)

InvestList = sorted(InvestList)

#Rebuld Investor >> Site List
Invest_Site = {}
Invest_Count = {}
for Site in Site_Invest:
    name = Site[1]
    if Invest_Site.has_key(name):
        Invest_Site[name] += '|' + Site[0]
        Invest_Count[name] += Site[2]
    else:
        Invest_Site.update({name:Site[0]})
        Invest_Count.update({name:Site[2]})

Invest_Site_Count = []
for name in InvestList:
    Invest_Site_Count.append({'Name':name, 'Sites':Invest_Site[name], 'Count':Invest_Count[name]})


y = pd.DataFrame.from_dict(Invest_Site_Count)
y

,Count,Name,Sites
0,27,A. Chris Oishi,US-Dk1|US-Dk2|US-Dk3
1,6,Aaron Fellows,US-Rls|US-Rms|US-Rws
2,4,Abel Rodrigues,PT-Esp
3,7,Achim Grelle,SE-Asa|SE-Fla|SE-Kno|SE-Sk1
4,0,Adam Wolf,KZ-AL1|KZ-AL2|KZ-AL3|KZ-AL4|KZ-CW1|KZ-CW2|KZ-C...
5,9,Adrian Rocha,US-An1|US-An2|US-An3
6,0,Aikaterini Trepekli,GR-Vcs
7,2,Akira Miyata,JP-Aka|JP-Ksa|JP-Mas|JP-Onn|JP-Yaw
8,22,Alan Barr,CA-SF2|CA-SF3|CA-SJ1|CA-SJ2|CA-SJ3
9,0,Alan Knapp,US-Ra1|US-Ra2


### 1.3 Save processed data to disk

In [3]:
y.to_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/SiteInfo/Invest_Site_Count.csv')

## 2. Web of Science raw data processing
Web of Science full records directly download form WoS interface are important resources for bibliometrics analysis, while they are in the form of TAG + contents, it is not convinient for python to handle. We'll transform it into a more neat form, then save those beautiful new Records to disk for further processing. 

### 2.1 Load the data by Tags
The downloaded WoS full record data was coded in UTF-8, which contained \xef\xbb\xbf and TM codes in the head. Delete them! Here the original data is rather small, We'll dump in the Web of Science ISI data into our memory directly. Usually this process should be done Line by Line, putting all files into memory is not a good way in processing a great bunch of data. 

In [1]:
import pandas as pd

Fileinput = open('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/5654Records_Original.txt')

#Set Reocrds as a List and each reocord as a dict to save attributes
Records = []
record = {}
CurrentTag = ''

for line in Fileinput.readlines():
    #Delete BOM Data in front, TM symbol in middle and \n in the last
    line = line.replace('\xef\xbb\xbf','').replace('\xe2\x84\xa2','').replace('\n','')
    #Extraction
    if line[:2] != '  ' and len(line) > 0:
        CurrentTag = line[:2]
        if CurrentTag != 'ER':
            record.update({CurrentTag:[line[3:]]})
        elif line[:2] == 'ER':
            Records.append(record)
            record = {}
    elif line[:2] == '  ':
        record[CurrentTag].append(line[3:])
        
Fileinput.close()

### 2.2 New record structure
Now the raw data are already well saved in Records, and they could also be processed in python. All attributes are recorded as Lists in memory. For single value attributes, there is only one value in the list, we'll transform them back to normal string. While for multi-value atrributes like AF, AU and CR, values are  well listed in the data structure. 
Transport the data into a 2D chart(pandas.DataFrame), let's see how they look like.

In [2]:
for index,record in enumerate(Records):
    for attr in record:
        if isinstance(record[attr],list):
            temp = ''
            for line in record[attr]:
                temp += line + '|'
            Records[index][attr] = temp[:-1]
    
PRecords = pd.DataFrame.from_records(Records, index = range(len(Records)))
PRecords

,AB,AF,AR,AU,BE,BN,BP,BS,C1,CA,...,SU,TC,TI,U1,U2,UT,VL,VR,WC,Z9
0,"In this study, net surface radiation (R-n) was...","Mahalakshmi, D. V.|Paul, Arati|Dutta, D.|Ali, ...",NaN,"Mahalakshmi, DV|Paul, A|Dutta, D|Ali, MM|Dadhw...",NaN,NaN,1,NaN,"[Mahalakshmi, D. V.; Ali, M. M.; Dadhwal, V. K...",NaN,...,NaN,0,Estimation of net surface radiation using eddy...,1,1,WOS:000381162400001,33,1.0,Geochemistry & Geophysics,0
1,"To date, direct validation of city-wide emissi...","Vaughan, Adam R.|Lee, James D.|Misztal, Pawel ...",NaN,"Vaughan, AR|Lee, JD|Misztal, PK|Metzger, S|Sha...",NaN,NaN,455,NaN,"[Vaughan, Adam R.] Univ York, Dept Chem, York,...",NaN,...,NaN,3,Spatially resolved flux measurements of NOx fr...,8,10,WOS:000380099700022,189,NaN,"Chemistry, Physical",3
2,Large variability in N2O emissions from manage...,"Grant, Robert F.|Neftel, Albrecht|Calanca, Pie...",NaN,"Grant, RF|Neftel, A|Calanca, P",NaN,NaN,3549,NaN,"[Grant, Robert F.] Univ Alberta, Dept Renewabl...",NaN,...,NaN,0,Ecological controls on N2O emission in surface...,11,12,WOS:000379427700003,13,NaN,"Ecology; Geosciences, Multidisciplinary",0
3,"Conversions of natural ecosystems, e.g., from ...","Merten, Jennifer|Roell, Alexander|Guillaume, T...",5,"Merten, J|Roll, A|Guillaume, T|Meijide, A|Tari...",NaN,NaN,NaN,NaN,"[Merten, Jennifer; Dittrich, Christoph; Faust,...",NaN,...,NaN,2,Water scarcity and oil palm expansion: social ...,16,28,WOS:000380049100006,21,NaN,Ecology; Environmental Studies,2
4,A scheme describing the process of stream-aqui...,"Zeng, Yujin|Xie, Zhenghui|Yu, Yan|Liu, Shuang|...",NaN,"Zeng, YJ|Xie, ZH|Yu, Y|Liu, S|Wang, LY|Jia, BH...",NaN,NaN,2333,NaN,"[Zeng, Yujin; Xie, Zhenghui; Liu, Shuang; Wang...",NaN,...,NaN,3,Ecohydrological effects of stream-aquifer wate...,10,15,WOS:000379419500013,20,NaN,"Geosciences, Multidisciplinary; Water Resources",3
5,There have been few studies conducted on the c...,"Yang, Zesu|Zhang, Qiang|Hao, Xiaocui",6809749,"Yang, ZS|Zhang, Q|Hao, XC",NaN,NaN,NaN,NaN,"[Yang, Zesu] Chengdu Univ Informat Technol, Co...",NaN,...,NaN,0,Evapotranspiration Trend and Its Relationship ...,8,8,WOS:000379433600001,NaN,NaN,Meteorology & Atmospheric Sciences,0
6,The lifetime of nitrogen oxides (NOx) affects ...,"Romer, Paul S.|Duffey, Kaitlin C.|Wooldridge, ...",NaN,"Romer, PS|Duffey, KC|Wooldridge, PJ|Allen, HM|...",NaN,NaN,7623,NaN,"[Romer, Paul S.; Duffey, Kaitlin C.; Wooldridg...",NaN,...,NaN,2,The lifetime of nitrogen oxides in an isoprene...,16,26,WOS:000379417300009,16,NaN,Meteorology & Atmospheric Sciences,2
7,"The emission, dispersion, and photochemistry o...","Su, Luping|Patton, Edward G.|de Arellano, Jord...",NaN,"Su, LP|Patton, EG|de Arellano, JVG|Guenther, A...",NaN,NaN,7725,NaN,"[Su, Luping; Mak, John E.] SUNY Stony Brook, S...",NaN,...,NaN,3,Understanding isoprene photooxidation using ob...,7,10,WOS:000379417300016,16,NaN,Meteorology & Atmospheric Sciences,3
8,"We measured volatile organic compounds (VOCs),...","Rantala, Pekka|Jarvi, Leena|Taipale, Risto|Lau...",NaN,"Rantala, P|Jarvi, L|Taipale, R|Laurila, TK|Pat...",NaN,NaN,7981,NaN,"[Rantala, Pekka; Jarvi, Leena; Taipale, Risto;...",NaN,...,NaN,0,Anthropogenic and biogenic influence on VOC fl...,3,12,WOS:000379417300032,16,NaN,Meteorology & Atmospheric Sciences,0
9,The dry component of total nitrogen and sulfur...,"Rumsey, Ian C.|Walker, John T.",NaN,"Rumsey, IC|Walker, JT",NaN,NaN,2581,NaN,"[Rumsey, Ian C.] Coll Charleston, Dept Phys & ...",NaN,...,NaN,0,Application of an online ion-chromatography-ba...,4,10,WOS:000379397100008,9,NaN,Meteorology & Atmospheric Sciences,0


### 2.3 Data output
Just output the whole dataframe to one single xlsx file using the function pandas.DataFrame.to_excel and pandas.DataFrame.to_csv().

In [9]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

writer = pd.ExcelWriter('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/Full_Record_WoS.xlsx')
PRecords.to_excel(writer, 'Sheet1')
writer.save()

In [11]:
PRecords.to_csv('D:/_Research/Project_Sharing_Data_FromLinux/ProjectRebuild_2017/Data/Full_Record_WoS.csv')